In [1]:
import tensorflow
from Net import Net
import matplotlib.pyplot as plt
import numpy as np
import cv2
import SimpleITK as sitk


Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [2]:
net = Net('model/stable_10_07_20')


In [ ]:
mask = net.find_objects('images/121.tif')
contours = net.process_mask(mask)
net.make_json(contours)

In [3]:
net.process_file('images/121.tif')


In [3]:
net.process_directory('images/')

file images//121.tif already processed
file images//122.tif already processed


In [ ]:
import base64
import json
import os.path as osp
import PIL.Image

from labelme import __version__
import io
from labelme.logger import logger
from labelme import PY2
from labelme import QT4
from labelme import utils

def load_image_file(filename):
    try:
        image_pil = PIL.Image.open(filename)
    except IOError:
        logger.error('Failed opening image file: {}'.format(filename))
        return

    # apply orientation to image according to exif
    image_pil = utils.apply_exif_orientation(image_pil)

    with io.BytesIO() as f:
        ext = osp.splitext(filename)[1].lower()
        if PY2 and QT4:
            format = 'PNG'
        elif ext in ['.jpg', '.jpeg']:
            format = 'JPEG'
        else:
            format = 'PNG'
        image_pil.save(f, format=format)
        f.seek(0)
        return f.read()

def load(filename):
    keys = [
        'version',
        'imageData',
        'imagePath',
        'shapes',  # polygonal annotations
        'flags',   # image level flags
        'imageHeight',
        'imageWidth',
    ]
    shape_keys = [
        'label',
        'points',
        'group_id',
        'shape_type',
        'flags',
    ]

    with open(filename, 'r') as f:
        data = json.load(f)
    version = data.get('version')

    if data['imageData'] is not None:
        imageData = base64.b64decode(data['imageData'])
    else:
        # relative path from label file to relative path from cwd
        imagePath = osp.join(osp.dirname(filename), data['imagePath'])
        imageData = load_image_file(imagePath)
    flags = data.get('flags') or {}
    imagePath = data['imagePath']

    shapes = [
        dict(
            label=s['label'],
            points=s['points'],
            shape_type=s.get('shape_type', 'polygon'),
            flags=s.get('flags', {}),
            group_id=s.get('group_id'),
            other_data={
                k: v for k, v in s.items() if k not in shape_keys
            }
        )
        for s in data['shapes']
    ]

    otherData = {}
    for key, value in data.items():
        if key not in keys:
            otherData[key] = value



In [ ]:
tmp = load('images/121.json')

In [11]:
def mathing(image, mask):
    fixed_image_itk = sitk.GetImageFromArray(image)
    moving_image = sitk.GetImageFromArray(mask)

    initial_transform = sitk.CenteredTransformInitializer(fixed_image_itk,
                                                          moving_image,
                                                          sitk.Euler2DTransform(),
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)
    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)  # REGULAR
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetMetricAsCorrelation()

    registration_method.SetInterpolator(sitk.sitkLinear)

    # Optimizer settings.
    # registration_method.SetOptimizerAsRegularStepGradientDescent(learningRate=0.5, numberOfIterations=300, minStep=0.01)
    #                                                       # convergenceMinimumValue=1e-9,
    #                                                        #convergenceWindowSize=100)
    registration_method.SetOptimizerAsGradientDescent(learningRate=0.0001, numberOfIterations=10000000,
                                                           convergenceMinimumValue=1e-9,
                                                           convergenceWindowSize=50)
    registration_method.SetOptimizerScalesFromPhysicalShift()
    # registration_method.SetOptimizerAsConjugateGradientLineSearch(learningRate=1, numberOfIterations=300, convergenceMinimumValue=1e-9, convergenceWindowSize=100)

    # Setup for the multi-resolution framework.
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    final_transform = registration_method.Execute(sitk.Cast(fixed_image_itk , sitk.sitkFloat32),
                                                  sitk.Cast(moving_image, sitk.sitkFloat32))

    moving_resampled = sitk.Resample(moving_image, fixed_image_itk, final_transform, sitk.sitkLinear, 0.0,
                                     moving_image.GetPixelID())
    final_image = sitk.GetArrayViewFromImage(moving_resampled)

    return np.asarray(final_image, 'uint8')
